<a href="https://colab.research.google.com/github/alilotfi90/Advanced-Deep-Learning-Project/blob/main/MobileNet_opt_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install efficientnet_pytorch
from zipfile import ZipFile
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet

# Extract the training and test data
train_zip = '/content/drive/MyDrive/images/train2.zip'
test_zip = '/content/drive/MyDrive/images/test2.zip'
with ZipFile(train_zip, 'r') as zip:
    zip.extractall()
with ZipFile(test_zip, 'r') as zip:
    zip.extractall()

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16444 sha256=ab585347860f2056b0aac3c4d859e0afca4d24021800f250a226ff6cf8caffe0
  Stored in directory: /root/.cache/pip/wheels/29/16/24/752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
Successfully built efficientnet_pytorch


In [9]:
IMG_SIZE = 224
BATCH_SIZE = 32

In [10]:
train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.85, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [11]:
train_dataset = ImageFolder('/content/train', transform=train_transform)
valid_dataset = ImageFolder('/content/test', transform=valid_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Define the model
base_model = models.mobilenet_v2(pretrained=True)
base_model = nn.Sequential(*list(base_model.children())[:-1])

In [12]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.base_model = base_model
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(1280, 100)  # Assuming you have 100 classes

    def forward(self, x):
        x = self.base_model(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = MyModel()
# model.cuda()

In [13]:
# Training the model
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)



In [14]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Training the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=2, verbose=True)




In [15]:
for num_epochs in num_epochs_list:
  print("for max number epoch", num_epochs)

  for epoch in range(num_epochs):
      running_loss = 0.0
      epoch_loss = []
      for i, data in enumerate(train_loader, 0):
          inputs, labels = data
          inputs, labels = inputs.to(device), labels.to(device)
          optimizer.zero_grad()

          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
          epoch_loss.append(loss.item())
          if i % 100 == 99:
              print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
              running_loss = 0.0

      # Update the learning rate scheduler
      scheduler.step(sum(epoch_loss) / len(epoch_loss))

  print('Finished training')

  # Calculate accuracy
  correct = 0
  total = 0
  with torch.no_grad():
      for data in valid_loader:
          images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))


for max number epoch 10
[1,   100] loss: 4.255
[2,   100] loss: 3.292
[3,   100] loss: 2.727
[4,   100] loss: 2.356
[5,   100] loss: 2.077
[6,   100] loss: 1.815
[7,   100] loss: 1.711
[8,   100] loss: 1.546
[9,   100] loss: 1.372
[10,   100] loss: 1.265
Finished training
Accuracy of the network on the test images: 51 %
for max number epoch 20
[1,   100] loss: 1.204
[2,   100] loss: 1.113
[3,   100] loss: 1.010
[4,   100] loss: 0.918
[5,   100] loss: 0.872
[6,   100] loss: 0.816
[7,   100] loss: 0.776
[8,   100] loss: 0.729
[9,   100] loss: 0.693
[10,   100] loss: 0.602
[11,   100] loss: 0.573
[12,   100] loss: 0.554
[13,   100] loss: 0.504
[14,   100] loss: 0.554
[15,   100] loss: 0.483
[16,   100] loss: 0.484
[17,   100] loss: 0.433
[18,   100] loss: 0.445
[19,   100] loss: 0.424
[20,   100] loss: 0.436
Finished training
Accuracy of the network on the test images: 67 %
for max number epoch 40
[1,   100] loss: 0.395
[2,   100] loss: 0.392
[3,   100] loss: 0.377
[4,   100] loss: 0.377


In [20]:
torch.save(model.state_dict(), '/content/drive/MyDrive/images/mobile_best_weights1')